<a href="https://colab.research.google.com/github/dp-by-dt/animator/blob/main/animation_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

import zipfile
import io

for zip_name in uploaded.keys():
    with zipfile.ZipFile(io.BytesIO(uploaded[zip_name]), 'r') as zip_ref:
        zip_ref.extractall('train_log')


Saving RIFEv4.25_0919.zip to RIFEv4.25_0919.zip


In [ ]:
# ============================================
# RIFE Frame Interpolation Tool - Part 1
# For Hand-Drawn Animation Interpolation
# ============================================

# CELL 1: Setup and Installation
# ================================
print("🎬 Installing RIFE and dependencies...")
print("This will take 2-3 minutes on first run.\n")

!git clone https://github.com/hzwer/Practical-RIFE.git
%cd Practical-RIFE

!pip install -q torch torchvision torchaudio
!pip install -q opencv-python-headless
!pip install -q tqdm
!pip install -q pillow
!pip install -q gdown

print("\n✅ Installation complete!")
print("📦 Downloading RIFE v4.25 model (recommended for animations)...")
print("This may take 1-2 minutes...\n")

# Create train_log directory
!mkdir -p train_log

# Download model v4.25 from Google Drive (file ID from the README)
!gdown 1tJKvMBvvbAYRByOpHKc6KrmLKzF4-qpL -O train_log.zip
!unzip -q train_log.zip -d ./
!rm train_log.zip

print("\n✅ Model downloaded and extracted successfully!")



🎬 Installing RIFE and dependencies...
This will take 2-3 minutes on first run.

Cloning into 'Practical-RIFE'...
remote: Enumerating objects: 625, done.
remote: Counting objects: 100% (385/385), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 625 (delta 372), reused 248 (delta 248), pack-reused 240 (from 2)
Receiving objects: 100% (625/625), 3.04 MiB | 8.48 MiB/s, done.
Resolving deltas: 100% (384/384), done.
/content/Practical-RIFE

✅ Installation complete!
📦 Downloading RIFE v4.25 model (recommended for animations)...
This may take 1-2 minutes...

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1tJKvMBvvbAYRByOpHKc6KrmLKzF4-qpL

but Gdown can't. Please check connections and permis

In [ ]:

# CELL 2: Import Libraries and Setup
# ===================================
import os
import cv2
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
import shutil
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json

# Create working directories
os.makedirs('keyframes', exist_ok=True)
os.makedirs('output_frames', exist_ok=True)
os.makedirs('final_videos', exist_ok=True)

print("✅ Libraries imported and directories created!")



✅ Libraries imported and directories created!


In [ ]:

# CELL 3: Upload Keyframes
# =========================
print("📤 Upload your keyframes (JPG/PNG)")
print("Tip: Name them in order (e.g., frame_001.jpg, frame_002.jpg, etc.)\n")

# Clear previous keyframes
for f in os.listdir('keyframes'):
    os.remove(os.path.join('keyframes', f))

uploaded = files.upload()

# Save uploaded files
for filename, content in uploaded.items():
    with open(f'keyframes/{filename}', 'wb') as f:
        f.write(content)

# Get sorted list of keyframes
keyframe_files = sorted([f for f in os.listdir('keyframes') if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
num_keyframes = len(keyframe_files)

print(f"\n✅ Uploaded {num_keyframes} keyframes:")
for i, kf in enumerate(keyframe_files, 1):
    print(f"  {i}. {kf}")



📤 Upload your keyframes (JPG/PNG)
Tip: Name them in order (e.g., frame_001.jpg, frame_002.jpg, etc.)



Saving frame_001.jpg to frame_001 (5).jpg
Saving frame_002.jpg to frame_002 (5).jpg

✅ Uploaded 2 keyframes:
  1. frame_001 (5).jpg
  2. frame_002 (5).jpg


In [ ]:

# CELL 4: Configure Timestamps
# =============================
print("\n⚙️ TIMESTAMP CONFIGURATION\n")
print("Choose your timestamp mode:\n")

mode_selector = widgets.RadioButtons(
    options=['Auto-distribute (Equal spacing)', 'Manual (Assign each timestamp)'],
    description='Mode:',
    disabled=False
)

display(mode_selector)

# Auto-distribute settings
print("\n📊 Auto-distribute settings:")
video_duration = widgets.FloatText(
    value=10.0,
    description='Video Duration (seconds):',
    disabled=False,
    style={'description_width': '200px'}
)

display(video_duration)

# Manual timestamp input area
print("\n✏️ Manual timestamp assignment:")
print("(Only used if Manual mode is selected)")

manual_timestamps_text = widgets.Textarea(
    value='',
    placeholder='Enter timestamps for each frame, one per line\nExample:\n0.0\n2.5\n5.0\n7.5\n10.0',
    description='Timestamps:',
    layout=widgets.Layout(width='500px', height='150px'),
    style={'description_width': '100px'}
)

display(manual_timestamps_text)

# FPS setting
fps_input = widgets.IntText(
    value=24,
    description='FPS:',
    disabled=False
)

print("\n🎞️ Frame rate setting:")
display(fps_input)

# Generate button
generate_button = widgets.Button(
    description='📋 Process Timestamps',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

output_area = widgets.Output()

def process_timestamps(b):
    with output_area:
        clear_output()

        global keyframe_timestamps, fps_value
        fps_value = fps_input.value

        if mode_selector.value.startswith('Auto'):
            # Auto-distribute mode
            duration = video_duration.value
            keyframe_timestamps = np.linspace(0, duration, num_keyframes).tolist()
            print(f"✅ Auto-distributed {num_keyframes} frames over {duration} seconds\n")
        else:
            # Manual mode
            manual_input = manual_timestamps_text.value.strip().split('\n')
            try:
                keyframe_timestamps = [float(t.strip()) for t in manual_input if t.strip()]
                if len(keyframe_timestamps) != num_keyframes:
                    print(f"❌ ERROR: You provided {len(keyframe_timestamps)} timestamps but have {num_keyframes} frames!")
                    return
                print(f"✅ Manual timestamps assigned\n")
            except ValueError:
                print("❌ ERROR: Invalid timestamp format. Use numbers only (e.g., 0.0, 2.5, 5.0)")
                return

        # Display timeline
        print("📍 TIMELINE PREVIEW:\n")
        for i, (kf, ts) in enumerate(zip(keyframe_files, keyframe_timestamps)):
            print(f"  [{ts:6.2f}s] {kf}")

        total_duration = max(keyframe_timestamps)
        total_frames = int(total_duration * fps_value)

        print(f"\n📊 Video Stats:")
        print(f"  • Total Duration: {total_duration:.2f} seconds")
        print(f"  • FPS: {fps_value}")
        print(f"  • Total Frames to Generate: {total_frames}")
        print(f"  • Segments to Interpolate: {num_keyframes - 1}")

        print("\n✅ Ready to generate! Proceed to next cell.")

generate_button.on_click(process_timestamps)
display(generate_button)
display(output_area)




⚙️ TIMESTAMP CONFIGURATION

Choose your timestamp mode:



RadioButtons(description='Mode:', options=('Auto-distribute (Equal spacing)', 'Manual (Assign each timestamp)'…


📊 Auto-distribute settings:


FloatText(value=10.0, description='Video Duration (seconds):', style=DescriptionStyle(description_width='200px…


✏️ Manual timestamp assignment:
(Only used if Manual mode is selected)


Textarea(value='', description='Timestamps:', layout=Layout(height='150px', width='500px'), placeholder='Enter…


🎞️ Frame rate setting:


IntText(value=24, description='FPS:')

Button(button_style='success', description='📋 Process Timestamps', layout=Layout(height='40px', width='200px')…

Output()

In [ ]:

# CELL 5: Load RIFE Model
# ========================
print("🔧 Loading RIFE model...\n")

import sys
sys.path.insert(0, '/content/Practical-RIFE')

# Check if model files exist
import os
model_path = '/content/Practical-RIFE/train_log'
if not os.path.exists(model_path):
    print("❌ Model directory not found!")
    print("Please ensure Cell 1 ran successfully and model was downloaded.")
else:
    print(f"✓ Model directory found: {model_path}")
    print(f"✓ Contents: {os.listdir(model_path)[:5]}...")  # Show first 5 files

# Import RIFE model
try:
    from model.RIFE_HDv3 import Model
    print("✓ Successfully imported RIFE_HDv3\n")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Trying alternative import method...\n")
    import importlib.util
    spec = importlib.util.spec_from_file_location("RIFE_HDv3", "/content/Practical-RIFE/train_log/RIFE_HDv3.py")
    rife_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(rife_module)
    Model = rife_module.Model
    print("✓ Alternative import successful\n")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
model.load_model('/content/Practical-RIFE/train_log', -1)
model.eval()
model.device()

print(f"✅ RIFE model loaded successfully!")
print(f"   Device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
else:
    print("   ⚠️ Running on CPU (slower, but will work)")



🔧 Loading RIFE model...

✓ Model directory found: /content/Practical-RIFE/train_log
✓ Contents: ['RIFE_HDv3.py', 'flownet.pkl', 'train_log', 'IFNet_HDv3.py', '__MACOSX']...
❌ Import error: No module named 'model.RIFE_HDv3'
Trying alternative import method...

✓ Alternative import successful

✅ RIFE model loaded successfully!
   Device: cuda
   GPU: Tesla T4


In [ ]:

# CELL 6: Frame Interpolation Engine
# ===================================
def load_image(path, target_height=720):
    """Load and resize image to target resolution"""
    img = cv2.imread(path)
    h, w = img.shape[:2]

    # Resize to target height, maintain aspect ratio
    scale = target_height / h
    new_w = int(w * scale)
    new_h = target_height

    # Make dimensions divisible by 32 (RIFE requirement)
    new_w = (new_w // 32) * 32
    new_h = (new_h // 32) * 32

    img = cv2.resize(img, (new_w, new_h))
    return img

def interpolate_frames(img1, img2, num_frames, model, device):
    """Generate intermediate frames between two images using RIFE"""

    # Convert to torch tensors
    img1_tensor = torch.from_numpy(img1).permute(2, 0, 1).float().unsqueeze(0) / 255.0
    img2_tensor = torch.from_numpy(img2).permute(2, 0, 1).float().unsqueeze(0) / 255.0

    img1_tensor = img1_tensor.to(device)
    img2_tensor = img2_tensor.to(device)

    interpolated = []

    with torch.no_grad():
        for i in range(1, num_frames + 1):
            timestep = i / (num_frames + 1)

            # RIFE interpolation
            middle = model.inference(img1_tensor, img2_tensor, timestep)

            # Convert back to numpy
            middle_np = (middle[0].permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
            interpolated.append(middle_np)

    return interpolated

print("✅ Interpolation engine ready!")



✅ Interpolation engine ready!


In [ ]:

# CELL 7: Generate Video
# =======================
print("\n🎬 GENERATING ANIMATED VIDEO\n")
print("=" * 60)

# Clear previous output
for f in os.listdir('output_frames'):
    os.remove(os.path.join('output_frames', f))

# Load all keyframes
keyframes_data = []
target_height = 720  # 720p

print("📥 Loading keyframes...")
for kf_file in keyframe_files:
    img = load_image(f'keyframes/{kf_file}', target_height)
    keyframes_data.append(img)
    print(f"  ✓ {kf_file} - {img.shape[1]}x{img.shape[0]}")

height, width = keyframes_data[0].shape[:2]
print(f"\n📐 Output resolution: {width}x{height}\n")

# Generate all frames
frame_count = 0
all_frames = []

print("🎨 Generating intermediate frames...\n")

for seg_idx in range(len(keyframes_data) - 1):
    img1 = keyframes_data[seg_idx]
    img2 = keyframes_data[seg_idx + 1]

    ts1 = keyframe_timestamps[seg_idx]
    ts2 = keyframe_timestamps[seg_idx + 1]

    duration = ts2 - ts1
    num_frames_between = int(duration * fps_value) - 1

    print(f"Segment {seg_idx + 1}/{len(keyframes_data) - 1}:")
    print(f"  {keyframe_files[seg_idx]} [{ts1:.2f}s] → {keyframe_files[seg_idx + 1]} [{ts2:.2f}s]")
    print(f"  Duration: {duration:.2f}s | Frames to generate: {num_frames_between}")

    # Add first keyframe
    all_frames.append(img1)
    cv2.imwrite(f'output_frames/frame_{frame_count:06d}.png', img1)
    frame_count += 1

    # Generate intermediate frames
    if num_frames_between > 0:
        interpolated = interpolate_frames(img1, img2, num_frames_between, model, device)

        for inter_frame in tqdm(interpolated, desc="  Interpolating"):
            all_frames.append(inter_frame)
            cv2.imwrite(f'output_frames/frame_{frame_count:06d}.png', inter_frame)
            frame_count += 1

    print(f"  ✓ Generated {num_frames_between + 1} frames\n")

# Add final keyframe
all_frames.append(keyframes_data[-1])
cv2.imwrite(f'output_frames/frame_{frame_count:06d}.png', keyframes_data[-1])
frame_count += 1

print(f"\n✅ Total frames generated: {frame_count}")
print("\n📹 Creating MP4 video...\n")

# Create video
output_video_path = 'final_videos/animation.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps_value, (width, height))

for frame in tqdm(all_frames, desc="Writing video"):
    video_writer.write(frame)

video_writer.release()

print(f"\n✅ VIDEO CREATED SUCCESSFULLY!")
print(f"📁 Location: {output_video_path}")
print(f"📊 Stats:")
print(f"   • Duration: {max(keyframe_timestamps):.2f} seconds")
print(f"   • Total Frames: {frame_count}")
print(f"   • FPS: {fps_value}")
print(f"   • Resolution: {width}x{height}")




🎬 GENERATING ANIMATED VIDEO

📥 Loading keyframes...
  ✓ frame_001 (5).jpg - 960x704
  ✓ frame_002 (5).jpg - 960x704

📐 Output resolution: 960x704

🎨 Generating intermediate frames...

Segment 1/1:
  frame_001 (5).jpg [0.00s] → frame_002 (5).jpg [10.00s]
  Duration: 10.00s | Frames to generate: 239


  Interpolating: 100%|██████████| 239/239 [00:07<00:00, 30.89it/s]


  ✓ Generated 240 frames


✅ Total frames generated: 241

📹 Creating MP4 video...



Writing video: 100%|██████████| 241/241 [00:01<00:00, 151.71it/s]


✅ VIDEO CREATED SUCCESSFULLY!
📁 Location: final_videos/animation.mp4
📊 Stats:
   • Duration: 10.00 seconds
   • Total Frames: 241
   • FPS: 24
   • Resolution: 960x704


In [ ]:

# CELL 8: Download Video
# =======================
print("\n⬇️ DOWNLOAD YOUR ANIMATION\n")

from google.colab import files
files.download('final_videos/animation.mp4')

print("✅ Download started!")
print("\n💡 TIP: You can re-run from Cell 3 to create another video with different keyframes.")


⬇️ DOWNLOAD YOUR ANIMATION



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started!

💡 TIP: You can re-run from Cell 3 to create another video with different keyframes.
